# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-07-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-06-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-07 04:23:40,33.93911,67.709953,39486,1467,32977,5042.0,Afghanistan,101.432527,3.715241
1,NaN,NaN,NaN,Albania,2020-10-07 04:23:40,41.15330,20.168300,14568,403,8965,5200.0,Albania,506.220029,2.766337
2,NaN,NaN,NaN,Algeria,2020-10-07 04:23:40,28.03390,1.659600,52399,1768,36763,13868.0,Algeria,119.493167,3.374110
3,NaN,NaN,NaN,Andorra,2020-10-07 04:23:40,42.50630,1.521800,2370,53,1615,702.0,Andorra,3067.365560,2.236287
4,NaN,NaN,NaN,Angola,2020-10-07 04:23:40,-11.20270,17.873900,5725,211,2598,2916.0,Angola,17.419075,3.685590


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39227,39233,39254,39268,39285,39290,39297,39341,39422,39486
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,13259,13391,13518,13649,13806,13965,14117,14266,14410,14568
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,51067,51213,51368,51530,51690,51847,51995,52136,52270,52399


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1453,1455,1458,1458,1458,1458,1462,1462,1466,1467
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,377,380,384,387,388,389,392,396,400,403
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1714,1719,1726,1736,1741,1749,1756,1760,1768,1768


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32642,32642,32746,32789,32842,32842,32842,32852,32879,32977
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7397,7629,7732,7847,8077,8342,8536,8675,8825,8965
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35860,35962,36063,36174,36282,36385,36482,36578,36672,36763


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5526,5588,5606,5640,5997,6024,6048,6073,6085,6116
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,885,886,886,896,898,902,921,921,921,923


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,25,25,27,27,28,27,27,27,27,27
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,50,50,50,52,53,53,53,53,53,55
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-10-07 04:23:40,32.539527,-86.644082,1839,27,0,1812.0,"Autauga, Alabama, US",3291.628631,1.468189
669,1075.0,Lamar,Alabama,US,2020-10-07 04:23:40,33.779950,-88.096680,403,2,0,401.0,"Lamar, Alabama, US",2919.232162,0.496278
670,1077.0,Lauderdale,Alabama,US,2020-10-07 04:23:40,34.901719,-87.656247,1851,37,0,1814.0,"Lauderdale, Alabama, US",1996.139288,1.998920


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7499341,210886,2952390
India,6757131,104555,5744693
Brazil,4969141,147494,4430340


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7499341,210886,2952390,4336065,2020-10-07 04:23:40,37.923092,-91.345036
India,6757131,104555,5744693,907883,2020-10-07 04:23:40,23.088275,81.806127
Brazil,4969141,147494,4430340,391307,2020-10-07 04:23:40,-12.669522,-48.480493
Russia,1231277,21559,984767,224951,2020-10-07 04:23:40,54.546312,62.120860
Colombia,869808,27017,770812,71979,2020-10-07 04:23:40,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,838202,16244,0
Texas,798569,16528,0
Florida,720125,14767,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,838202,16244,0,821958,2020-10-07 04:23:40,37.843962,-120.728594
Texas,798569,16528,0,782041,2020-10-07 04:23:40,31.660643,-98.653069
Florida,720125,14767,0,705358,2020-10-07 04:23:40,28.940755,-82.700744
New York,466908,33219,0,433689,2020-10-07 04:23:40,42.544151,-75.474183
Georgia,324650,7229,0,317421,2020-10-07 04:23:40,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,275856,6681,0
Florida,Miami-Dade,172849,3349,0
Illinois,Cook,149588,5268,0
Texas,Harris,148769,2632,0
Arizona,Maricopa,143917,3431,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,275856,6681,0,269175,2020-10-07 04:23:40,34.308284,-118.228241,6037.0
Florida,Miami-Dade,172849,3349,0,169500,2020-10-07 04:23:40,25.611236,-80.551706,12086.0
Illinois,Cook,149588,5268,0,144320,2020-10-07 04:23:40,41.841448,-87.816588,17031.0
Texas,Harris,148769,2632,0,146137,2020-10-07 04:23:40,29.858649,-95.393395,48201.0
Arizona,Maricopa,143917,3431,0,140486,2020-10-07 04:23:40,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,39341,14266,52136,2110,5402,107,798486,52496,27148,48146,...,505619,2145,58612,78434,1096,41498,10,2041,15052,7888
2020-10-05,39422,14410,52270,2370,5530,107,809728,52677,27173,48896,...,518222,2155,58946,79117,1097,41957,10,2041,15089,7898
2020-10-06,39486,14568,52399,2370,5725,107,824468,53083,27181,49819,...,532779,2177,59343,79796,1098,42432,10,2047,15170,7915


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,1462,396,1760,53,195,3,21018,977,894,813,...,42440,48,480,653,35,330,1,591,333,228
2020-10-05,1466,400,1768,53,199,3,21468,984,895,818,...,42459,48,485,658,35,339,1,592,334,228
2020-10-06,1467,403,1768,53,211,3,21827,990,897,822,...,42535,49,489,665,35,349,1,593,335,229


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,32852,8675,36578,1540,2577,96,636672,44672,24888,38629,...,2403,1844,55281,68917,1020,34698,8,1320,14187,6359
2020-10-05,32879,8825,36672,1615,2591,96,649017,44710,24890,39058,...,2409,1862,55633,69832,1022,35182,8,1323,14306,6424
2020-10-06,32977,8965,36763,1615,2598,96,660272,44932,24915,39790,...,2417,1878,56058,70719,1023,35599,8,1327,14313,6440


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,370,377,382,382,386,393,397,400,402,403
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1723,1740,1751,1759,1772,1787,1807,1815,1832,1851
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,621,628,634,636,644,647,652,652,662,676
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5927,5945,5971,6017,6043,6068,6117,6141,6156,6173


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,159169,8405,0,220754,87013,832713,73054,58297,21243,49,...,201210,790194,77618,1785,1327,151642,89874,16635,132663,6504
2020-10-05,159713,8613,0,221070,87430,835507,73515,59120,21363,49,...,203699,794319,78723,1817,1322,152322,90276,16749,134359,6629
2020-10-06,160477,8752,0,221934,88071,838202,74169,59241,21466,49,...,205375,798569,79439,1821,1321,152945,90663,16942,136379,6770


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-04        1828    6073     921  686   1656     613    926    3679   
2020-10-05        1831    6085     921  687   1662     613    927    3705   
2020-10-06        1839    6116     923  691   1665     616    928    3730   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-04         1205      639  ...     265     44      324      123   
2020-10-05         1208      642  ...     268     47      335      123   
2020-10-06         1210      644  ...     272     51      346      127   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-04            361   616   360          0      119     42  
2020-10-05            363   625   365          0      121     44  
2020-10-06            371   629   368          0      120     46  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,2558,58,0,5706,1425,16147,2068,4513,645,0,...,2577,16320,478,58,20,3270,2142,364,1377,53
2020-10-05,2559,58,0,5707,1447,16171,2069,4517,645,0,...,2597,16437,482,58,20,3273,2158,367,1381,53
2020-10-06,2580,58,0,5713,1469,16244,2081,4521,646,0,...,2621,16528,488,58,20,3288,2165,370,1399,53


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-04          27      53       7   10     15      15     40      44   
2020-10-05          27      53       7   10     15      15     40      44   
2020-10-06          27      55       7   10     15      16     40      45   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-04           42       13  ...       2      1        4        1   
2020-10-05           42       13  ...       2      1        4        1   
2020-10-06           42       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-04              2     1     2          0        6      0  
2020-10-05              2     1     2          0        6      0  
2020-10-06              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.001120,0.010555,0.002712,0.000000,0.005959,0.0,0.009696,0.010997,0.000479,0.015053,...,0.047581,0.010839,0.006422,0.010149,0.000000,0.010224,0.0,0.00000,0.005209,0.000380
2020-10-05,0.002059,0.010094,0.002570,0.123223,0.023695,0.0,0.014079,0.003448,0.000921,0.015578,...,0.024926,0.004662,0.005698,0.008708,0.000912,0.011061,0.0,0.00000,0.002458,0.001268
2020-10-06,0.001623,0.010965,0.002468,0.000000,0.035262,0.0,0.018204,0.007707,0.000294,0.018877,...,0.028090,0.010209,0.006735,0.008582,0.000912,0.011321,0.0,0.00294,0.005368,0.002152


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.000000,0.010204,0.002278,0.0,0.010363,0.0,0.010724,0.005144,0.000000,0.004944,...,0.000778,0.000000,0.006289,0.006163,0.0,0.003040,0.0,0.003396,0.000000,0.000000
2020-10-05,0.002736,0.010101,0.004545,0.0,0.020513,0.0,0.021410,0.007165,0.001119,0.006150,...,0.000448,0.000000,0.010417,0.007657,0.0,0.027273,0.0,0.001692,0.003003,0.000000
2020-10-06,0.000682,0.007500,0.000000,0.0,0.060302,0.0,0.016723,0.006098,0.002235,0.004890,...,0.001790,0.020833,0.008247,0.010638,0.0,0.029499,0.0,0.001689,0.002994,0.004386


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.000304,0.016284,0.002631,0.000000,0.057882,0.0,0.016863,0.001996,0.000965,0.015350,...,0.002922,0.007100,0.007784,0.012027,0.000000,0.037558,0.0,0.000000,0.006599,0.005058
2020-10-05,0.000822,0.017291,0.002570,0.048701,0.005433,0.0,0.019390,0.000851,0.000080,0.011106,...,0.002497,0.009761,0.006367,0.013277,0.001961,0.013949,0.0,0.002273,0.008388,0.010222
2020-10-06,0.002981,0.015864,0.002481,0.000000,0.002702,0.0,0.017342,0.004965,0.001004,0.018741,...,0.003321,0.008593,0.007639,0.012702,0.000978,0.011853,0.0,0.003023,0.000489,0.002491


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,0.004982,0.022879,NaN,0.001611,0.005640,0.003848,0.007183,0.000000,0.005586,0.0,...,0.008091,0.003517,0.018275,0.003937,0.000000,0.007033,0.005088,0.009344,0.014259,0.021838
2020-10-05,0.003418,0.024747,NaN,0.001431,0.004792,0.003355,0.006310,0.014117,0.005649,0.0,...,0.012370,0.005220,0.014236,0.017927,-0.003768,0.004484,0.004473,0.006853,0.012784,0.019219
2020-10-06,0.004784,0.016138,NaN,0.003908,0.007332,0.003226,0.008896,0.002047,0.004821,0.0,...,0.008228,0.005350,0.009095,0.002201,-0.000756,0.004090,0.004287,0.011523,0.015034,0.021270


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-04      0.005501  0.004134  0.000000  0.011799  0.000604  0.000000   
2020-10-05      0.001641  0.001976  0.000000  0.001458  0.003623  0.000000   
2020-10-06      0.004369  0.005094  0.002172  0.005822  0.001805  0.004894   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-04      0.002165  0.005191  0.000831  0.000000  ...  0.019231   
2020-10-05      0.001080  0.007067  0.002490  0.004695  ...  0.011321   
2020-10-06      0.001079  0.006748  0.001656  0.003115  ...  0.014925   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-04      0.073171  0.018868  0.008197   0.011204  0.023256  0.002786   
2020-10-05      0.068182  0.033951  0.000000   0.005540  0.014610  0.013889   
2020-10-06      0.085106  0.032836  0.032520   0.022039  0.006400  0.008219   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-04            NaN  0.008475  0.024390  
2020-10-05            NaN  0.016807  0.047619  
2020-10-06            NaN -0.008264  0.045455  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,0.000000,0.0,NaN,0.000175,0.012793,0.001986,0.003883,0.000000,0.00000,NaN,...,0.006641,0.001534,0.004202,0.0,0.0,0.000918,0.000000,0.002755,0.003644,0.0
2020-10-05,0.000391,0.0,NaN,0.000175,0.015439,0.001486,0.000484,0.000886,0.00000,NaN,...,0.007761,0.007169,0.008368,0.0,0.0,0.000917,0.007470,0.008242,0.002905,0.0
2020-10-06,0.008206,0.0,NaN,0.001051,0.015204,0.004514,0.005800,0.000886,0.00155,NaN,...,0.009241,0.005536,0.012448,0.0,0.0,0.004583,0.003244,0.008174,0.013034,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                 \
Admin2         Autauga   Baldwin Barbour Bibb Blount   Bullock Butler   
2020-10-04         0.0  0.000000     0.0  0.0    0.0  0.000000    0.0   
2020-10-05         0.0  0.000000     0.0  0.0    0.0  0.000000    0.0   
2020-10-06         0.0  0.037736     0.0  0.0    0.0  0.066667    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-10-04      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-05      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-06      0.022727      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-04          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-05          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-06          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.000673,0.010753,0.002836,0.005609,0.015798,0.008679,0.013131,0.010532,0.000495,0.017328,...,0.034418,0.011488,0.008220,0.010489,0.000458,0.009684,2.739100e-33,0.000389,0.005640,0.001338
2020-10-05,0.001366,0.010423,0.002703,0.064416,0.019746,0.004339,0.013605,0.006990,0.000708,0.016453,...,0.029672,0.008075,0.006959,0.009598,0.000685,0.010372,1.369550e-33,0.000194,0.004049,0.001303
2020-10-06,0.001495,0.010694,0.002586,0.032208,0.027504,0.002170,0.015904,0.007349,0.000501,0.017665,...,0.028881,0.009142,0.006847,0.009090,0.000798,0.010847,6.847751e-34,0.001567,0.004709,0.001728


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.000734,0.008028,0.003179,1.067522e-18,0.015076,8.417643e-53,0.023540,0.004893,0.001071,0.005010,...,0.001047,0.000338,0.006775,0.008419,9.259740e-12,0.011584,0.0,0.002125,0.000189,0.000036
2020-10-05,0.001735,0.009064,0.003862,5.337611e-19,0.017795,4.208821e-53,0.022475,0.006029,0.001095,0.005580,...,0.000747,0.000169,0.008596,0.008038,4.629870e-12,0.019428,0.0,0.001909,0.001596,0.000018
2020-10-06,0.001208,0.008282,0.001931,2.668805e-19,0.039048,2.104411e-53,0.019599,0.006063,0.001665,0.005235,...,0.001269,0.010501,0.008422,0.009338,2.314935e-12,0.024463,0.0,0.001799,0.002295,0.002202


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-04,0.000346,0.020855,0.002709,0.013618,0.069224,0.008037,0.017405,0.003036,0.001084,0.018641,...,0.003041,0.006853,0.009467,0.013044,0.001474,0.027312,1.604942e-35,0.004400,0.005749,0.004007
2020-10-05,0.000584,0.019073,0.002640,0.031159,0.037328,0.004018,0.018398,0.001943,0.000582,0.014874,...,0.002769,0.008307,0.007917,0.013160,0.001717,0.020631,8.024708e-36,0.003336,0.007068,0.007114
2020-10-06,0.001782,0.017468,0.002561,0.015580,0.020015,0.002009,0.017870,0.003454,0.000793,0.016807,...,0.003045,0.008450,0.007778,0.012931,0.001348,0.016242,4.012354e-36,0.003180,0.003779,0.004803


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,0.006740,0.019768,NaN,0.002165,0.007880,0.004047,0.008136,0.001662,0.006789,5.296183e-62,...,0.006993,0.005018,0.016181,0.004539,0.000614,0.006657,0.005997,0.010903,0.018202,0.022318
2020-10-05,0.005079,0.022257,NaN,0.001798,0.006336,0.003701,0.007223,0.007890,0.006219,2.648092e-62,...,0.009682,0.005119,0.015209,0.011233,-0.001577,0.005571,0.005235,0.008878,0.015493,0.020768
2020-10-06,0.004931,0.019198,NaN,0.002853,0.006834,0.003463,0.008060,0.004968,0.005520,1.324046e-62,...,0.008955,0.005235,0.012152,0.006717,-0.001167,0.004830,0.004761,0.010200,0.015264,0.021019


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-04      0.005470  0.008011  0.006383  0.008681  0.003308  0.000815   
2020-10-05      0.003555  0.004994  0.003191  0.005069  0.003466  0.000408   
2020-10-06      0.003962  0.005044  0.002681  0.005446  0.002635  0.002651   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-04      0.002229  0.006777  0.005159  0.005304  ...  0.016873   
2020-10-05      0.001655  0.006922  0.003824  0.004999  ...  0.014097   
2020-10-06      0.001367  0.006835  0.002740  0.004057  ...  0.014511   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater    Teton     Uinta   
2020-10-04      0.058726  0.021063  0.014915   0.016349  0.01771  0.002975   
2020-10-05      0.063454  0.027507  0.007457   0.010945  0.01616  0.008432   
2020-10-06      0.074280  0.030171  0.019989   0.016492  0.01128  0.008326   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-04      -0.571428  0.007631  0.027530  
2020-10-05      -0.571428  0.012219  0.037574  
2020-10-06      -0.571428  0.001977  0.041514  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-04,0.001475,0.005941,NaN,0.001445,0.011529,0.003318,0.002694,0.000129,0.002443,NaN,...,0.010423,0.003419,0.007569,1.533788e-20,0.000411,0.003583,0.000925,0.005114,0.007676,0.003755
2020-10-05,0.000933,0.002970,NaN,0.000810,0.013484,0.002402,0.001589,0.000508,0.001222,NaN,...,0.009092,0.005294,0.007969,7.668939e-21,0.000206,0.002250,0.004197,0.006678,0.005291,0.001878
2020-10-06,0.004570,0.001485,NaN,0.000931,0.014344,0.003458,0.003694,0.000697,0.001386,NaN,...,0.009167,0.005415,0.010208,3.834469e-21,0.000103,0.003417,0.003720,0.007426,0.009162,0.000939


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-10-04     -0.000894  0.002462  1.189791e-15 -0.002555  0.000018   
2020-10-05     -0.000447  0.001231  5.948954e-16 -0.001278  0.000009   
2020-10-06     -0.000224  0.019484  2.974477e-16 -0.000639  0.000005   

Province_State                                                        ...  \
Admin2           Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-10-04     -0.001414  0.000407  0.000133  9.537947e-08 -0.000145  ...   
2020-10-05     -0.000707  0.000204  0.000066  4.768973e-08 -0.000072  ...   
2020-10-06      0.032980  0.000102  0.011397  2.384487e-08 -0.000036  ...   

Province_State       Wyoming                                                   \
Admin2                  Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-10-04      3.814697e-06    0.0  0.000006      0.0        0.0   0.0   0.0   
2020-10-05      1.907349e-06    0.0  0.000003      0.0        0.0   0.0   0.0   
2020-10-06      9.536743e-07    0.0  0.000001      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-10-04      -0.999969      0.0    NaN  
2020-10-05      -0.999969      0.0    NaN  
2020-10-06      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201007,AK,9861,NaN,480213,NaN,490074,46.0,NaN,NaN,...,490074,1,0,1a883e2869ef7b3a3c422abb9a41a55364b84ae5,0,0,0,0,0,NaN
1,20201007,AL,161418,19126.0,1038526,NaN,1180818,777.0,17915.0,NaN,...,1199944,21,216,b677706c829d281111e03e6b30854b2b3f106914,0,0,0,0,0,NaN
2,20201007,AR,88880,3966.0,1003441,NaN,1087671,538.0,5740.0,237.0,...,1092321,35,82,3c2e6abdabaccdd357b1c2fd6d7fec6738e065b3,0,0,0,0,0,NaN
3,20201007,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,...,1616,0,0,993f0592cbe6f9ba149c02096a4b96e1279caad8,0,0,0,0,0,NaN
4,20201007,AZ,222538,4792.0,1300948,NaN,1518694,681.0,20050.0,147.0,...,1523486,20,63,0a9c9654f4ee87e874ee29587cd1591984c9547c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201007,AK,9861.0,NaN,480213.0,NaN,490074.0,46.0,NaN,NaN,...,490074,1,0,1a883e2869ef7b3a3c422abb9a41a55364b84ae5,0,0,0,0,0,NaN
1,20201007,AL,161418.0,19126.0,1038526.0,NaN,1180818.0,777.0,17915.0,NaN,...,1199944,21,216,b677706c829d281111e03e6b30854b2b3f106914,0,0,0,0,0,NaN
2,20201007,AR,88880.0,3966.0,1003441.0,NaN,1087671.0,538.0,5740.0,237.0,...,1092321,35,82,3c2e6abdabaccdd357b1c2fd6d7fec6738e065b3,0,0,0,0,0,NaN
3,20201007,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,...,1616,0,0,993f0592cbe6f9ba149c02096a4b96e1279caad8,0,0,0,0,0,NaN
4,20201007,AZ,222538.0,4792.0,1300948.0,NaN,1518694.0,681.0,20050.0,147.0,...,1523486,20,63,0a9c9654f4ee87e874ee29587cd1591984c9547c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-07,AK,9861,NaN,480213,NaN,490074,46.0,NaN,NaN,NaN,...,490074,1,0,1a883e2869ef7b3a3c422abb9a41a55364b84ae5,0,0,0,0,0,NaN
2020-10-07,AL,161418,19126.0,1038526,NaN,1180818,777.0,17915.0,NaN,1867.0,...,1199944,21,216,b677706c829d281111e03e6b30854b2b3f106914,0,0,0,0,0,NaN
2020-10-07,AR,88880,3966.0,1003441,NaN,1087671,538.0,5740.0,237.0,NaN,...,1092321,35,82,3c2e6abdabaccdd357b1c2fd6d7fec6738e065b3,0,0,0,0,0,NaN
2020-10-07,AS,0,NaN,1616,NaN,1616,NaN,NaN,NaN,NaN,...,1616,0,0,993f0592cbe6f9ba149c02096a4b96e1279caad8,0,0,0,0,0,NaN
2020-10-07,AZ,222538,4792.0,1300948,NaN,1518694,681.0,20050.0,147.0,NaN,...,1523486,20,63,0a9c9654f4ee87e874ee29587cd1591984c9547c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-07,AK,9861.0,NaN,480213.0,NaN,490074.0,46.0,NaN,NaN,NaN,...,490074,1,0,1a883e2869ef7b3a3c422abb9a41a55364b84ae5,0,0,0,0,0,NaN
2020-10-07,AL,161418.0,19126.0,1038526.0,NaN,1180818.0,777.0,17915.0,NaN,1867.0,...,1199944,21,216,b677706c829d281111e03e6b30854b2b3f106914,0,0,0,0,0,NaN
2020-10-07,AR,88880.0,3966.0,1003441.0,NaN,1087671.0,538.0,5740.0,237.0,NaN,...,1092321,35,82,3c2e6abdabaccdd357b1c2fd6d7fec6738e065b3,0,0,0,0,0,NaN
2020-10-07,AS,0.0,NaN,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,...,1616,0,0,993f0592cbe6f9ba149c02096a4b96e1279caad8,0,0,0,0,0,NaN
2020-10-07,AZ,222538.0,4792.0,1300948.0,NaN,1518694.0,681.0,20050.0,147.0,NaN,...,1523486,20,63,0a9c9654f4ee87e874ee29587cd1591984c9547c,0,0,0,0,0,NaN
